# Pain Anamnesis Classification using Neural Networks

This notebook implements a deep neural network for classifying pain symptoms based on biomechanical measurements. The model predicts multiple binary pain indicators across different body regions.

## Setup and Data Preparation

The following cell:
1. Imports required libraries
2. Loads and preprocesses the pain anamnesis data
3. Prepares features and targets for model training

### Data Structure:
- **Input Features**: 7 biomechanical measurements including:
  - Left/Right movement deviation averages
  - Left/Right resting deviation averages
  - Left/Right step averages
  - Shoe size
- **Target Variables**: Binary pain indicators (0/1) for:
  - Various foot regions (forefoot, midfoot, heel)
  - Upper limb regions (wrist, elbow, fingers, upper arm, thumb, forearm)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Load the Excel file (make sure you have openpyxl installed: pip install openpyxl)
data = pd.read_excel("colored_columns_output_filtered.xlsx")  # Replace with your file path

# Define the 7 input columns.
input_cols = [
    "SchiefstandBewegungMmDurchschnitt_links",
    "SchiefstandBewegungMmDurchschnitt_rechts",
    "SchiefstandRuheMmDurchschnitt_links",
    "SchiefstandRuheMmDurchschnitt_rechts",
    "AuftrittDurchschnitt_links",
    "AuftrittDurchschnitt_rechts",
    "Schuhgröße"
]

# Define the original binary target columns.
binary_target_cols = [
    "Schmerz_Vorfuß_Links", "Schmerz_Vorfuß_Rechts", 
    "Schmerz_Mittelfuß_Links", "Schmerz_Mittelfuß_Rechts",
    "Schmerz_Ferse_Links", "Schmerz_Ferse_Rechts",
    "Schmerz_Handgelenk_links", "Schmerz_Handgelenk_rechts",	
    "Schmerz_Ellenbogen_links", "Schmerz_Ellenbogen_rechts",	
    "Schmerz_Finger_links", "Schmerz_Finger_rechts",	
    "Schmerz_Oberarm_links", "Schmerz_Oberarm_rechts",	
    "Schmerz_Daumen_links", "Schmerz_Daumen_rechts",	
    "Schmerz_Unterarm_links", "Schmerz_Unterarm_rechts"
]

# Identify ordinal columns (all "Schmerz_*" columns not in the binary list).
all_schmerz_cols = [col for col in data.columns if col.startswith("Schmerz_")]
ordinal_target_cols = [col for col in all_schmerz_cols if col not in binary_target_cols]

# Clean data.
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(subset=ordinal_target_cols, inplace=True)

# Convert original binary targets to 0/1.
for col in binary_target_cols:
    data[col] = data[col].map({True: 1, 'TRUE': 1, 'True': 1,
                               False: 0, 'FALSE': 0, 'False': 0})

# Ensure ordinal targets are integers.
for col in ordinal_target_cols:
    data[col] = data[col].astype(int)

# Convert ordinal targets to binary: set to 0 if value is 0–3, else 1.
for col in ordinal_target_cols:
    data[col] = (data[col] > 3).astype(int)

# Combine original binary targets and the converted ordinal targets.
all_binary_target_cols = binary_target_cols + ordinal_target_cols

# Extract features and combined binary targets.
X = data[input_cols].values.astype(np.float32)
y_all = data[all_binary_target_cols].values.astype(np.float32)

# Split data into train (70%), validation (15%), and test (15%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y_all, test_size=0.15, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=42
)  # ≈15% for validation

# Scale the input features.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## Custom Dataset Implementation

The `PainDataset` class handles data management for PyTorch training:

1. **Features**:
   - Converts numpy arrays to PyTorch tensors
   - Scales input features using StandardScaler
   - Handles batch processing

2. **Labels**:
   - Converts multiple pain indicators to binary format
   - Manages multi-target classification

3. **Data Loading**:
   - Implements required PyTorch Dataset methods
   - Enables efficient batch processing during training

In [ ]:
class PainDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)  # binary targets
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset instances.
train_dataset = PainDataset(X_train, y_train)
val_dataset   = PainDataset(X_val, y_val)
test_dataset  = PainDataset(X_test, y_test)

# Create DataLoaders.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Neural Network Architecture

The `BinaryModel` class implements a deep neural network with:

1. **Structure**:
   - 7 hidden layers with batch normalization
   - Decreasing layer sizes: 128 → 128 → 64 → 64 → 32 → 32 → 16
   - Multiple binary outputs for pain classification

2. **Features**:
   - LeakyReLU activation functions
   - Dropout regularization (0.2)
   - Batch normalization for stable training
   - Multi-target binary classification output

3. **Design Choices**:
   - Deep architecture for complex pattern recognition
   - Regularization to prevent overfitting
   - Batch normalization for faster convergence

In [ ]:
class BinaryModel(nn.Module):
    def __init__(self, input_dim, num_targets):
        super(BinaryModel, self).__init__()
        # Define 7 hidden layers. Feel free to adjust the hidden sizes.
        self.hidden1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.hidden2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.hidden3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.hidden4 = nn.Linear(64, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.hidden5 = nn.Linear(64, 32)
        self.bn5 = nn.BatchNorm1d(32)
        self.hidden6 = nn.Linear(32, 32)
        self.bn6 = nn.BatchNorm1d(32)
        self.hidden7 = nn.Linear(32, 16)
        self.bn7 = nn.BatchNorm1d(16)
        # Output layer predicts all binary targets.
        self.out = nn.Linear(16, num_targets)
        # Define dropout (you may adjust dropout rates if needed).
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.hidden1(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn2(self.hidden2(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn3(self.hidden3(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn4(self.hidden4(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn5(self.hidden5(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn6(self.hidden6(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn7(self.hidden7(x)))
        x = self.dropout(x)
        logits = self.out(x)
        return logits

num_targets = len(all_binary_target_cols)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BinaryModel(input_dim=7, num_targets=num_targets).to(device)

## Model Training Configuration

The training process implements:

1. **Loss Function**:
   - Binary Cross-Entropy with Logits
   - Handles multiple binary classifications simultaneously

2. **Optimization**:
   - AdamW optimizer with learning rate 0.001
   - Learning rate scheduling with ReduceLROnPlateau
   - Gradient clipping for stable training

3. **Training Process**:
   - 50 epochs with early stopping
   - Validation-based model checkpointing
   - Accuracy and loss monitoring

In [3]:
criterion = nn.BCEWithLogitsLoss()  # Binary loss

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                       factor=0.5, patience=5, verbose=True)

num_epochs = 50

def binary_accuracy(preds, targets):
    pred_labels = (torch.sigmoid(preds) > 0.5).float()
    correct = (pred_labels == targets).float()
    return correct.mean().item()


# Cell 6: Training Loop with Validation and Gradient Clipping
best_val_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0
    total_train_acc = 0.0
    num_train_samples = 0
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        batch_size = X_batch.size(0)
        total_train_loss += loss.item() * batch_size
        total_train_acc += binary_accuracy(logits, y_batch) * batch_size
        num_train_samples += batch_size
        
    avg_train_loss = total_train_loss / num_train_samples
    avg_train_acc = total_train_acc / num_train_samples
    
    # Validation evaluation.
    model.eval()
    total_val_loss = 0.0
    total_val_acc = 0.0
    num_val_samples = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            logits = model(X_batch)
            loss = criterion(logits, y_batch)
            batch_size = X_batch.size(0)
            total_val_loss += loss.item() * batch_size
            total_val_acc += binary_accuracy(logits, y_batch) * batch_size
            num_val_samples += batch_size
            
    avg_val_loss = total_val_loss / num_val_samples
    avg_val_acc = total_val_acc / num_val_samples
    
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"Train Acc: {avg_train_acc:.4f} | Val Acc: {avg_val_acc:.4f}")
    
    scheduler.step(avg_val_loss)
    
    # Save the model if validation loss improves.
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        print("Validation loss improved, saving model.")

/opt/anaconda3/envs/ap/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50 | Train Loss: 0.6850 | Val Loss: 0.6554
Train Acc: 0.5700 | Val Acc: 0.6700
Validation loss improved, saving model.
Epoch 2/50 | Train Loss: 0.6273 | Val Loss: 0.6044
Train Acc: 0.6704 | Val Acc: 0.7392
Validation loss improved, saving model.
Epoch 3/50 | Train Loss: 0.5846 | Val Loss: 0.5642
Train Acc: 0.7292 | Val Acc: 0.7629
Validation loss improved, saving model.
Epoch 4/50 | Train Loss: 0.5536 | Val Loss: 0.5408
Train Acc: 0.7585 | Val Acc: 0.7808
Validation loss improved, saving model.
Epoch 5/50 | Train Loss: 0.5373 | Val Loss: 0.5235
Train Acc: 0.7694 | Val Acc: 0.7840
Validation loss improved, saving model.
Epoch 6/50 | Train Loss: 0.5290 | Val Loss: 0.5108
Train Acc: 0.7726 | Val Acc: 0.7855
Validation loss improved, saving model.
Epoch 7/50 | Train Loss: 0.5208 | Val Loss: 0.5055
Train Acc: 0.7733 | Val Acc: 0.7855
Validation loss improved, saving model.
Epoch 8/50 | Train Loss: 0.5154 | Val Loss: 0.5016
Train Acc: 0.7758 | Val Acc: 0.7856
Validation loss improved

## Model Inference and Evaluation

This section demonstrates the model's prediction capabilities:

1. **Inference Process**:
   - Loads the best model state
   - Processes test data
   - Generates binary predictions

2. **Output Format**:
   - Binary predictions (0/1) for each pain indicator
   - Probability thresholding at 0.5
   - Multiple simultaneous predictions per input

3. **Visualization**:
   - Displays sample predictions
   - Shows prediction format for clinical interpretation

In [4]:
model.eval()
with torch.no_grad():
    X_new = torch.tensor(X_test, dtype=torch.float32).to(device)
    logits = model(X_new)
    binary_preds = torch.sigmoid(logits)
    # Convert probabilities to binary predictions.
    binary_preds = (binary_preds > 0.5).float().cpu().numpy()
    
    print("Binary predictions (first 5):")
    print(binary_preds[:5])

Binary predictions (first 5):
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
